In [1]:
import json
import sys
import asyncio
from pathlib import Path

# Добавляем корневую директорию проекта в пути импорта
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Импортируем необходимые модули
from src.parsers.resume_extractor import ResumeExtractor
from src.llm_update_resume.llm_resume_updater import LLMResumeUpdater
from src.models.gap_analysis_models import ResumeTailoringAnalysis

# Пути к файлам
resume_file_path = Path("tests/fetched_resume_6d807532ff0ed6b79f0039ed1f63386d724a62.json")
gap_analysis_path = Path("gap_analysis_result.json")  # Путь к сохраненному результату gap-анализа

# Проверяем наличие файлов
print(f"Резюме файл существует: {resume_file_path.exists()}")
print(f"Gap-анализ файл существует: {gap_analysis_path.exists()}")

Резюме файл существует: True
Gap-анализ файл существует: True


In [3]:
# Загружаем данные
try:
    with open(resume_file_path, 'r', encoding='utf-8') as f:
        resume_data = json.load(f)
    print("Данные резюме успешно загружены")
except Exception as e:
    print(f"Ошибка при загрузке данных резюме: {e}")
    resume_data = None

try:
    with open(gap_analysis_path, 'r', encoding='utf-8') as f:
        gap_analysis_data = json.load(f)
    print("Данные gap-анализа успешно загружены")
except Exception as e:
    print(f"Ошибка при загрузке данных gap-анализа: {e}")
    gap_analysis_data = None

# Парсим данные резюме
resume_parser = ResumeExtractor()
parsed_resume = resume_parser.extract_resume_info(resume_data) if resume_data else None
parsed_resume_dict = parsed_resume.model_dump() if parsed_resume else None

# Создаем объект ResumeGapAnalysis из JSON
gap_result = ResumeTailoringAnalysis.model_validate(gap_analysis_data) if gap_analysis_data else None

Данные резюме успешно загружены
Данные gap-анализа успешно загружены


In [4]:
# Загружаем данные
with open(resume_file_path, 'r', encoding='utf-8') as f:
    resume_data = json.load(f)
print("Данные резюме успешно загружены")

with open(gap_analysis_path, 'r', encoding='utf-8') as f:
    gap_analysis_data = json.load(f)
print("Данные gap-анализа успешно загружены")

# Парсим данные резюме
resume_parser = ResumeExtractor()
parsed_resume = resume_parser.extract_resume_info(resume_data)
parsed_resume_dict = parsed_resume.model_dump()

# Создаем объект ResumeGapAnalysis из JSON
gap_result = ResumeTailoringAnalysis.model_validate(gap_analysis_data)

print("✅ Данные резюме и gap-анализа успешно загружены и валидированы")

Данные резюме успешно загружены
Данные gap-анализа успешно загружены
✅ Данные резюме и gap-анализа успешно загружены и валидированы


In [5]:
# Создаем экземпляр LLMResumeUpdater
resume_updater = LLMResumeUpdater()

# Получаем промпт для финального рерайта резюме
prompt = resume_updater._create_final_rewrite_prompt(parsed_resume_dict, gap_result)

# Выводим полный промпт
print("\n=== ПОЛНЫЙ ПРОМПТ ДЛЯ ФИНАЛЬНОГО РЕРАЙТА РЕЗЮМЕ ===")
print(prompt)

# Сохраняем промпт в файл для более удобного изучения
prompt_file_path = Path("formatted_rewrite_prompt.txt")
with open(prompt_file_path, 'w', encoding='utf-8') as f:
    f.write(prompt)
print(f"\nФорматированный промпт сохранен в файл: {prompt_file_path}")


=== ПОЛНЫЙ ПРОМПТ ДЛЯ ФИНАЛЬНОГО РЕРАЙТА РЕЗЮМЕ ===

        # ЗАДАЧА: УЛУЧШЕНИЕ РЕЗЮМЕ НА ОСНОВЕ GAP-АНАЛИЗА
        
        Твоя задача - переписать резюме соискателя, учитывая рекомендации из GAP-анализа, чтобы повысить его шансы на получение желаемой позиции.
        
        ## ТЕКУЩЕЕ РЕЗЮМЕ ДЛЯ АНАЛИЗА

### Желаемая должность
LLM Engineer

### Описание навыков (общее)
Я - увлеченный специалист, который видит потенциал искусственного интеллекта в упрощении жизни людей и бизнеса. Люблю копаться в LangChain и изучаю как работаю векторные базы данных. Считаю prompt engineering одним из основных навыков для создания грамотных LLM приложений.

### Ключевые навыки (список)
- ChatGPT
- AI
- prompt
- prompt engineering
- Python
- JSON API
- Нейро-сотрудники
- Умение работать в команде
- Искусственный интеллект
- Глубокое обучение
- NLP
- Машинное обучение
- LLM
- AI тренер
- LangChain
- Stable Diffusion
- Чат-бот

### Опыт работы
#### Опыт работы (ID: ОПЫТ_ID_1 [Компания не указана - A

In [6]:
# Асинхронная функция для выполнения запроса
async def run_resume_update():
    print("\n=== ЗАПУСК ФИНАЛЬНОГО РЕРАЙТА РЕЗЮМЕ ===")
    print("Отправляем запрос к LLM API...")
    
    try:
        # Вызываем метод update_resume для получения результата
        result = await resume_updater.update_resume(parsed_resume_dict, gap_result)
        
        if result:
            print("\n✅ Резюме успешно обновлено!")
            
            # # Выводим ключевые обновленные поля
            # print(f"\nНовая должность: {result.title}")
            # print("\nОбновленное описание навыков:")
            # print(result.skills)
            
            # print("\nОбновленные ключевые навыки:")
            # for skill in result.skill_set:
            #     print(f"- {skill}")
            
            # print("\nОбновленный опыт работы:")
            # for i, exp in enumerate(result.experience, 1):
            #     print(f"\nОпыт {i}:")
            #     print(f"Должность: {exp.position}")
            #     print(f"Описание: {exp.description}")
            
            # Выводим полный JSON результата
            print("\n=== ПОЛНЫЙ JSON РЕЗУЛЬТАТА ===")
            print(json.dumps(result.model_dump(), ensure_ascii=False, indent=2))
            
            # Сохраняем результат в файл
            result_path = Path("updated_resume_result.json")
            with open(result_path, "w", encoding="utf-8") as f:
                json.dump(result.model_dump(), f, ensure_ascii=False, indent=2)
            print(f"\nРезультат сохранен в файл: {result_path}")
            
            return result
        else:
            print("\n❌ Ошибка: Не удалось получить результат от LLM")
            return None
    
    except Exception as e:
        print(f"\n❌ Ошибка при выполнении финального рерайта: {e}")
        return None

# Запускаем асинхронную функцию
result = await run_resume_update()  # Для Jupyter нужно использовать top-level await


=== ЗАПУСК ФИНАЛЬНОГО РЕРАЙТА РЕЗЮМЕ ===
Отправляем запрос к LLM API...

✅ Резюме успешно обновлено!

=== ПОЛНЫЙ JSON РЕЗУЛЬТАТА ===
{
  "title": "LLM Engineer / AI Engineer с опытом работы с LangChain и RAG",
  "skills": "Специалист с более чем 3-летним опытом в сфере машинного обучения с акцентом на разработку и промышленное внедрение моделей LLM. Эксперт в PyTorch, LangChain и методах prompt engineering для создания эффективных AI-приложений. Практический опыт реализации сложных RAG-систем, fine-tuning моделей и организации процессов валидации с использованием современных фреймворков. Владение технологиями MLOps для развёртывания и поддержки моделей в продуктивной среде с Docker и GPU. Уверенное взаимодействие с Agile-командами, включая Data Architect, Data Engineer и Data Analyst. Развиты лидерские навыки управления командой и аналитические способности, необходимые для успешной реализации проектов AI-инфраструктуры.",
  "skill_set": [
    "PyTorch",
    "NLP",
    "LLM",
    "Lang